In [14]:
# from google.colab import drive
# drive.mount('/content/drive')

In [15]:
# %cd drive/MyDrive/analysis

In [16]:
import pandas as pd
import numpy as np

In [17]:
true_df = pd.read_csv('data/true_data.csv')
pred_df = pd.read_csv('data/pred_data.csv')

In [18]:
true_df.head()

,path,phase,TOE_x,TOE_y,HOSEL_x,HOSEL_y,GRIP_x,GRIP_y
0,sony_white_0006_0543,train,0.172222,0.320833,0.195370,0.304167,0.438889,0.344271
1,red_0008_0364,train,0.848958,0.387037,0.850000,0.425926,0.733333,0.557407
2,sasaki_green_1_0070,train,0.558333,0.343519,0.546875,0.335185,0.535417,0.412037
3,white_0008_0000,train,0.388542,0.892593,0.408854,0.896296,0.523438,0.624074
4,sony_red_0015_0436,train,0.970370,0.479167,0.947222,0.496875,0.664815,0.478125


In [19]:
true_train_df = true_df[true_df['phase']=='train'].copy()
pred_train_df = pred_df[pred_df['phase']=='train'].copy()

true_val_df = true_df[true_df['phase']=='val'].copy()
pred_val_df = pred_df[pred_df['phase']=='val'].copy()

In [20]:
true_val_df.head()

,path,phase,TOE_x,TOE_y,HOSEL_x,HOSEL_y,GRIP_x,GRIP_y
1809,frame_0153,val,0.752778,0.520833,0.731481,0.509375,0.482407,0.477083
1810,red_0712_0039_0051,val,0.475000,0.659375,0.468519,0.652604,0.482407,0.548438
1811,red_0712_0039_0050,val,0.494444,0.660417,0.484259,0.655208,0.484259,0.550000
1812,frame_0095,val,0.795370,0.477604,0.787037,0.472917,0.544444,0.445312
1813,frame_0139,val,0.499074,0.583333,0.490741,0.562500,0.444444,0.457813


In [21]:
def cal_PixelDistanceError(x_true_list, x_pred_list, y_true_list, y_pred_list, width, height):
    error = 0
    for x_true, x_pred, y_true, y_pred in zip(x_true_list, x_pred_list, y_true_list, y_pred_list):
        if np.isnan(x_true) or np.isnan(x_pred) or np.isnan(y_true) or np.isnan(y_pred):
            continue
        error += np.sqrt(
            width * (np.abs(x_true - x_pred)**2) + height * (np.abs(y_true - y_pred))
        )
    error /= len(x_true_list)
    return error

In [22]:
true_train_df['video'] = true_train_df['path'].apply(lambda x: '_'.join(x.split('_')[:-1]))
pred_train_df['video'] = pred_train_df['path'].apply(lambda x: '_'.join(x.split('_')[:-1]))
true_train_df['frame'] = true_train_df['path'].apply(lambda x: int(x.split('_')[-1]))
pred_train_df['frame'] = pred_train_df['path'].apply(lambda x: int(x.split('_')[-1]))

true_val_df['video'] = true_val_df['path'].apply(lambda x: '_'.join(x.split('_')[:-1]))
pred_val_df['video'] = pred_val_df['path'].apply(lambda x: '_'.join(x.split('_')[:-1]))
true_val_df['frame'] = true_val_df['path'].apply(lambda x: int(x.split('_')[-1]))
pred_val_df['frame'] = pred_val_df['path'].apply(lambda x: int(x.split('_')[-1]))

In [23]:
val_error_df = pd.DataFrame(columns=['video', 'frame', 'toe_error', 'toe_miss', 'toe_miss_rate', 'hosel_error', 'hosel_miss', 'hosel_miss_rate', 'grip_error', 'grip_miss', 'grip_miss_rate'])

width, height = 1080, 1920
for video in true_val_df['video'].unique():
    true_val_video_df = true_val_df[true_val_df['video']==video].copy()
    pred_val_video_df = pred_val_df[pred_val_df['video']==video].copy()
    true_val_video_df.sort_values('frame', inplace=True)
    pred_val_video_df.sort_values('frame', inplace=True)

    frame = len(true_val_video_df)
    
    toe_error = cal_PixelDistanceError(
        true_val_video_df['TOE_x'].values, pred_val_video_df['TOE_x'].values,
        true_val_video_df['TOE_y'].values, pred_val_video_df['TOE_y'].values,
        width, height
    )
    toe_miss = np.sum(np.isnan(true_val_video_df['TOE_x'].values) | np.isnan(pred_val_video_df['TOE_x'].values))
    toe_miss_rate = toe_miss / frame

    hosel_error = cal_PixelDistanceError(
        true_val_video_df['HOSEL_x'].values, pred_val_video_df['HOSEL_x'].values,
        true_val_video_df['HOSEL_y'].values, pred_val_video_df['HOSEL_y'].values,
        width, height
    )
    hosel_miss = np.sum(np.isnan(true_val_video_df['HOSEL_x'].values) | np.isnan(pred_val_video_df['HOSEL_x'].values))
    hosel_miss_rate = hosel_miss / frame

    grip_error = cal_PixelDistanceError(
        true_val_video_df['GRIP_x'].values, pred_val_video_df['GRIP_x'].values,
        true_val_video_df['GRIP_y'].values, pred_val_video_df['GRIP_y'].values,
        width, height
    )
    grip_miss = np.sum(np.isnan(true_val_video_df['GRIP_x'].values) | np.isnan(pred_val_video_df['GRIP_x'].values))
    grip_miss_rate = grip_miss / frame

    val_error_df.loc[len(val_error_df)] = [video, frame, toe_error, toe_miss, toe_miss_rate, hosel_error, hosel_miss, hosel_miss_rate, grip_error, grip_miss, grip_miss_rate]

In [24]:
val_error_df

,video,frame,toe_error,toe_miss,toe_miss_rate,hosel_error,hosel_miss,hosel_miss_rate,grip_error,grip_miss,grip_miss_rate
0,frame,165,2.030699,35,0.212121,2.030165,35,0.212121,1.813123,35,0.212121
1,red_0712_0039,236,1.942904,7,0.029661,1.818916,7,0.029661,1.587892,7,0.029661


In [25]:
train_error_df = pd.DataFrame(columns=['video', 'frame', 'toe_error', 'toe_miss', 'toe_miss_rate', 'hosel_error', 'hosel_miss', 'hosel_miss_rate', 'grip_error', 'grip_miss', 'grip_miss_rate'])

width, height = 1080, 1920
for video in true_train_df['video'].unique():
    true_train_video_df = true_train_df[true_train_df['video']==video].copy()
    pred_train_video_df = pred_train_df[pred_train_df['video']==video].copy()
    true_train_video_df.sort_values('frame', inplace=True)
    pred_train_video_df.sort_values('frame', inplace=True)

    frame = len(true_train_video_df)
    
    toe_error = cal_PixelDistanceError(
        true_train_video_df['TOE_x'].values, pred_train_video_df['TOE_x'].values,
        true_train_video_df['TOE_y'].values, pred_train_video_df['TOE_y'].values,
        width, height
    )
    toe_miss = np.sum(np.isnan(true_train_video_df['TOE_x'].values) | np.isnan(pred_train_video_df['TOE_x'].values))
    toe_miss_rate = toe_miss / frame

    hosel_error = cal_PixelDistanceError(
        true_train_video_df['HOSEL_x'].values, pred_train_video_df['HOSEL_x'].values,
        true_train_video_df['HOSEL_y'].values, pred_train_video_df['HOSEL_y'].values,
        width, height
    )
    hosel_miss = np.sum(np.isnan(true_train_video_df['HOSEL_x'].values) | np.isnan(pred_train_video_df['HOSEL_x'].values))
    hosel_miss_rate = hosel_miss / frame

    grip_error = cal_PixelDistanceError(
        true_train_video_df['GRIP_x'].values, pred_train_video_df['GRIP_x'].values,
        true_train_video_df['GRIP_y'].values, pred_train_video_df['GRIP_y'].values,
        width, height
    )
    grip_miss = np.sum(np.isnan(true_train_video_df['GRIP_x'].values) | np.isnan(pred_train_video_df['GRIP_x'].values))
    grip_miss_rate = grip_miss / frame

    train_error_df.loc[len(train_error_df)] = [video, frame, toe_error, toe_miss, toe_miss_rate, hosel_error, hosel_miss, hosel_miss_rate, grip_error, grip_miss, grip_miss_rate]

In [26]:
train_error_df

,video,frame,toe_error,toe_miss,toe_miss_rate,hosel_error,hosel_miss,hosel_miss_rate,grip_error,grip_miss,grip_miss_rate
0,sony_white_0006,123,1.874743,4,0.032520,1.880473,4,0.032520,1.622598,4,0.032520
1,red_0008,56,2.691813,0,0.000000,2.649470,0,0.000000,2.005761,0,0.000000
2,sasaki_green_1,60,1.895124,0,0.000000,2.287046,0,0.000000,1.800686,0,0.000000
3,white_0008,72,2.704008,1,0.013889,2.938291,1,0.013889,2.040370,1,0.013889
4,sony_red_0015,24,1.725761,1,0.041667,1.834701,1,0.041667,1.459979,1,0.041667
5,sony_red_0017,20,1.939999,1,0.050000,2.075431,1,0.050000,1.937947,1,0.050000
6,sony_red_0616_0002,33,1.914129,0,0.000000,2.198086,0,0.000000,1.751524,0,0.000000
7,white_0006,84,2.331992,1,0.011905,2.694228,1,0.011905,1.858376,1,0.011905
8,ebihara_green_1,50,2.892563,0,0.000000,2.751346,0,0.000000,1.936545,0,0.000000
9,green_0008,63,2.876156,1,0.015873,2.881368,1,0.015873,2.230684,1,0.015873


In [29]:
print(f"total: {train_error_df['frame'].sum()}")
print(f"total: {val_error_df['frame'].sum()}")

total: 1809
total: 401


In [27]:
train_error_df.to_csv('data/train_error.csv', index=False)
val_error_df.to_csv('data/val_error.csv', index=False)